In [96]:
import numpy as np
import random

from enums import CellState as cs
from enums import GameState as gs

class TicTacToe:
	def __init__(self, length = 3):
		self.length = length
		self.grid = np.full((self.length,self.length), cs.EMPTY.value)
		self.moves = [(i, j) for i in range(self.length) for j in range(self.length)]
		self.game_state = gs.PLAYING.value
		
	def reset(self):
		self.grid = np.full((self.length,self.length), cs.EMPTY.value)
		self.moves = [(i, j) for i in range(self.length) for j in range(self.length)]
		self.game_state = gs.PLAYING.value
		return self.grid

	# def check_winner(self):
	# 	winner_found = False

	# 	# Check for rows and columns
	# 	for i in range(self.length):
	# 		if np.all(self.grid[i, :] == cs.O.value) or np.all(self.grid[:, i] == cs.O.value):
	# 			self.game_state = cs.O.value
	# 			winner_found = True
	# 			break
			
	# 		if np.all(self.grid[i, :] == cs.X.value) or np.all(self.grid[:, i] == cs.X.value):
	# 			self.game_state = cs.X.value
	# 			winner_found = True
	# 			break
			
	# 	# Check for diagonals
	# 	if not winner_found:
	# 		for i in range(self.length):
	# 			if np.all(self.grid[i, i] == cs.O.value) or \
	# 				np.all(self.grid[i, self.length - i - 1] == cs.O.value):
	# 				self.game_state = cs.O.value
	# 				break

	# 			if np.all(self.grid[i, i] == cs.X.value) or \
	# 				np.all(self.grid[i, self.length - i - 1] == cs.X.value):
	# 				self.game_state = cs.X.value
	# 				break

	# 	return self.game_state

	def check_winner(self, grid):
		for row in grid:
			if row[0] == row[1] == row[2] and row[0] != 'EMPTY':
				return row[0]  # Return 'x' or 'o'

    # Check columns
		for col in range(3):
			if grid[0][col] == grid[1][col] == grid[2][col] and grid[0][col] != 'EMPTY':
				return grid[0][col]  # Return 'x' or 'o'

    # Check diagonals
		if grid[0][0] == grid[1][1] == grid[2][2] and grid[0][0] != 'EMPTY':
			return grid[0][0]  # Return 'x' or 'o'
		
		if grid[0][2] == grid[1][1] == grid[2][0] and grid[0][2] != 'EMPTY':
			return grid[0][2]  # Return 'x' or 'o'
		
		return gs.PLAYING.value  # No winner
	

	def step(self, player, move): 
		winner_reward = 10 #Kind of arbitrary and dependent on what we want to do.
		tie_winner_reward = 3 #Should be low, but not too low, since with optimal play, there is a tie.
		center_reward = 2
		corner_reward = 1
		remaining_reward = -0.5

		self.grid[move] = player #Set grid based on move.

		self.game_state = self.check_winner(self.grid) #Made chatGPT make this shitty ass function above.

		#Winning statement. Return grid and winner upon a winner being discovered, or upon a tie. Our "state" is both the grid and the winner.
		if self.game_state == gs.AGENT_WIN.value or self.game_state == gs.PLAYER_WIN.value:
			return self.grid, self.game_state, winner_reward, True
		elif not (cs.EMPTY.value in self.grid.flatten()): #If self.actions is EMPTY, this will be true.
			return self.grid, gs.TIE.value, tie_winner_reward if player == 1 else -tie_winner_reward, True
		else: #Here should go the rewards for particular actions, like putting in a center square. Maybe we start with no rewards at first?
			if move == (1,1):
				return self.grid, gs.PLAYING.value, center_reward, False
			elif move == (0,0) or move == (2,0) or move == (0,2) or move == (2,2):
				return self.grid, gs.PLAYING.value, corner_reward, False
			else:
				return self.grid, gs.PLAYING.value, remaining_reward, False
		
#That should be the end of the environment itself.
def array_to_tuple(array):
    """Convert an NxN array into a tuple of tuples."""
    return tuple(map(tuple, array))

def tuple_to_array(tpl):
    """Convert a tuple of tuples back into an NxN array."""
    return np.array(tpl)

class QLearningAgent:
	def __init__(self, env, learning_rate = 0.1, discount_factor=0.9, exploration_rate=0.2):
		self.env = env #Environment HAHA
		self.q_table = {} #Make dynamic list.
		self.learning_rate = learning_rate  # Alpha: learning rate
		self.discount_factor = discount_factor  # Gamma: discount factor for future rewards
		self.exploration_rate = exploration_rate  # Epsilon: exploration rate for epsilon-greedy
	
	def choose_move(self, state):
		if random.uniform(0, 1) < self.exploration_rate:
			available_moves_indexes = np.where(state.flatten() == cs.EMPTY.value)[0]
			new_moves = [self.env.moves[i] for i in available_moves_indexes]
			return random.choice(new_moves)
		else:
			# Choose the action with the highest Q-value
			return self.best_move(state)
	
	def valid_moves(self, state):
		available_moves_indexes = np.where(state.flatten() == cs.EMPTY.value)[0]
		return [self.env.moves[i] for i in available_moves_indexes]
	
	def best_move(self,state):
		new_moves = self.valid_moves(state)
		q_values = np.array([self.get_q_value(state,move) for move in new_moves])
		return new_moves[np.argmax(q_values)]
	
	def get_q_value(self,state, move):
		return self.q_table.get((array_to_tuple(state), move), 0)

	def update_q_value(self, state, move, reward, next_state):
		old_q_value = self.get_q_value(state,move)
		new_moves = self.valid_moves(next_state)
		next_best_q_value = max([self.get_q_value(next_state, newmove) for newmove in new_moves], default = 0)
		new_q_value = old_q_value + self.learning_rate * (reward + self.discount_factor * next_best_q_value - old_q_value)
		self.q_table[(array_to_tuple(state), move)] = new_q_value


#Training:
def opponent_moves(grid, moves):
	available_moves_indexes = np.where(grid.flatten() == cs.EMPTY.value)[0]
	new_moves = [moves[i] for i in available_moves_indexes]
	return random.choice(new_moves)


def train_q(episodes = 1000, print_interval = 1000):
	env = TicTacToe()
	agent = QLearningAgent(env)

	for episode in range(episodes):
		current_grid = agent.env.reset() + 0
		done = False
		
		while not done:
			move = agent.choose_move(current_grid)
			next_grid, winner, reward, done = agent.env.step(cs.O.value, move)
			agent.update_q_value(current_grid, move, reward, next_grid)

			if done:
				break
			
			current_grid = next_grid + 0

			opponent_move = opponent_moves(current_grid, agent.env.moves)
			next_grid, winner, reward, done = agent.env.step(cs.X.value, opponent_move)

			agent.update_q_value(current_grid, move, -reward, next_grid)

			if done:
				break

			current_grid = next_grid + 0
		
		if (episode + 1) % print_interval == 0:
			print(f"Episode {episode + 1}/{episodes} completed")

	return agent

printN = 10
N = printN*10000
ag = train_q(episodes=N, print_interval=int(N/printN))


Episode 10000/100000 completed
Episode 20000/100000 completed
Episode 30000/100000 completed
Episode 40000/100000 completed
Episode 50000/100000 completed
Episode 60000/100000 completed
Episode 70000/100000 completed
Episode 80000/100000 completed
Episode 90000/100000 completed
Episode 100000/100000 completed


In [97]:
ag.q_table[(((1, 0, 1), (2, 0, 2), (0, 0, 0)), (0, 1))]

9.202335569231275

In [98]:
index_values = [(i,j) for i in range(3) for j in range(3)]
index_values

[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]

In [99]:
position = [[1, 0, 2], [0, 1, 2], [0, 0, 0]]
position

[[1, 0, 2], [0, 1, 2], [0, 0, 0]]

In [100]:
for i in range(len(index_values)):
  try:
  	print(f"Position is: {position}, q-val for {index_values[i]} is: {ag.q_table[(array_to_tuple(position), index_values[i])]}")
  except:
    pass
   

Position is: [[1, 0, 2], [0, 1, 2], [0, 0, 0]], q-val for (0, 1) is: -0.4999992277308201
Position is: [[1, 0, 2], [0, 1, 2], [0, 0, 0]], q-val for (1, 0) is: -0.4999977851536227
Position is: [[1, 0, 2], [0, 1, 2], [0, 0, 0]], q-val for (2, 0) is: 0.9999998631085208
Position is: [[1, 0, 2], [0, 1, 2], [0, 0, 0]], q-val for (2, 1) is: -0.49999662422439073
Position is: [[1, 0, 2], [0, 1, 2], [0, 0, 0]], q-val for (2, 2) is: 9.999999999999993


In [118]:
import numpy as np

def print_board(board):
    """Prints the Tic-Tac-Toe board in a readable format."""
    symbols = {0: " ", 1: "O", 2: "X"}
    for row in board:
        print(" | ".join(symbols[cell] for cell in row))
        print("-" * 5)

def apply_moves(model, board, player):
    """Applies all available moves (empty spots, marked as 0) to the Tic-Tac-Toe board."""
    available_boards = []
    q_vals = []
    for i in range(3):
        for j in range(3):
            if board[i][j] == 0:  # Find an available spot
                new_board = board.copy()  # Copy the board
                new_board[i][j] = player  # Place an 'X' (represented by 2)
                available_boards.append(new_board)
                q_vals.append(model.q_table[(array_to_tuple(position), (i,j))])
    
    return available_boards, q_vals

# Example input board
input_board = np.array([[1, 0, 2], [0, 1, 2], [0, 0, 0]])

# Print the board before applying moves
print("Original board:")
print_board(input_board)

# Apply all available moves
available_boards, q_vals = apply_moves(ag, input_board, 1)

# Print the boards after applying the moves
for idx, board in enumerate(available_boards):
    print(f"\nBoard after move {idx + 1}:")
    print_board(board)
    print(f"q_value is: {q_vals[idx]}")

print()
print(f"Best move is move {np.argmax(q_vals)+1}!")


Original board:
O |   | X
-----
  | O | X
-----
  |   |  
-----

Board after move 1:
O | O | X
-----
  | O | X
-----
  |   |  
-----
q_value is: -0.4999992277308201

Board after move 2:
O |   | X
-----
O | O | X
-----
  |   |  
-----
q_value is: -0.4999977851536227

Board after move 3:
O |   | X
-----
  | O | X
-----
O |   |  
-----
q_value is: 0.9999998631085208

Board after move 4:
O |   | X
-----
  | O | X
-----
  | O |  
-----
q_value is: -0.49999662422439073

Board after move 5:
O |   | X
-----
  | O | X
-----
  |   | O
-----
q_value is: 9.999999999999993

Best move is move 5!
